# Brain Tumor MRI Detection

In [1]:
import tensorflow as tf

# TensorFlow automatically detects and uses the Metal GPU
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

GPU available: []


In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import os
import numpy as np
from PIL import Image

In [5]:
current_dir = os.getcwd()
print ("Current Directory: " + current_dir)

Current Directory: /Users/prathameshsanaye/Desktop/ml/brain_mri


In [7]:
train_dir = os.path.join(current_dir, 'Testing')
val_dir = os.path.join(current_dir, 'Training')

In [9]:
# Define the target size and batch size
target_size = (1250, 1250)
batch_size = 32


In [11]:
# Define the training and validation data generators
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')


Found 394 images belonging to 4 classes.
Found 2870 images belonging to 4 classes.


In [13]:
# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
                           input_shape=(target_size[0], target_size[1], 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [49]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,
    epochs=15,
    validation_data=val_generator,
    validation_steps=val_generator.samples//batch_size)

Epoch 1/15


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Saving The Model

In [17]:
# Save the trained model in the current directory
model.save(os.path.join(current_dir, 'model.h5'))

In [21]:
model = load_model('model.h5')

In [35]:
def prediction(predicted_no):
    if predicted_no==1:
        return 'giloma'
    elif predicted_no==2:
        return 'meningioma'
    elif predicted_no==3:
        return 'no tumor'
    elif predicted_no==4:
        return 'pitutary'
    else: return 'error'

In [45]:
# Load and preprocess your image
img_path = 'pitutuary.jpg'
img = image.load_img(img_path, target_size=(1250, 1250))  # Resize to 1250x1250
img_array = image.img_to_array(img)  # Convert to array
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize to [0, 1]

# Make a prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)

print(predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step
[1]
